# Quantitative Momentum Strategy

* source: https://youtu.be/xfzGZB4HhEE?si=p75PeszMBFeDVtiF&t=5927

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats # calculate percentile score, rank
import xlsxwriter
import yfinance as yf
import numpy as np

import time
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Mark up the beginning time 

# Start the timer
start_time = time.time()

# Print the current date and time in a human-readable format
current_time = datetime.fromtimestamp(start_time)
current_time_formatted = current_time.strftime("%Y-%m-%d %H:%M:%S")
print("Current date and time:", current_time_formatted)

Current date and time: 2025-02-21 16:48:13


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv("Data/sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

### Documentation for EOD free API: 
- Builk call (faster but can't make it work for now): https://eodhd.com/financial-apis/bulk-api-eod-splits-dividends
- one by one but limited to 20 : https://eodhd.com/financial-apis/api-for-historical-data-and-volumes

In [4]:
from config_api_file.secrets import EOD_API_TOKEN

In [5]:
# Trying first call one symbol with EOD
# be carefull on the limit

symbol = 'MCD'
exchange_id = 'US'
api_url = f'https://eodhd.com/api/eod/{symbol}.{exchange_id}?api_token={EOD_API_TOKEN}&fmt=json'
data = requests.get(api_url).json() 

MCD_US_data = data
MCD_US_data

[{'date': '2024-02-21',
  'open': 293.09,
  'high': 295.2,
  'low': 292.25,
  'close': 293.91,
  'adjusted_close': 286.9377,
  'volume': 2076811},
 {'date': '2024-02-22',
  'open': 294,
  'high': 296.67,
  'low': 293.245,
  'close': 295.92,
  'adjusted_close': 288.9,
  'volume': 3146598},
 {'date': '2024-02-23',
  'open': 296,
  'high': 298.74,
  'low': 295.07,
  'close': 297.75,
  'adjusted_close': 290.6866,
  'volume': 2199138},
 {'date': '2024-02-26',
  'open': 298.21,
  'high': 298.68,
  'low': 296.4,
  'close': 297.12,
  'adjusted_close': 290.0715,
  'volume': 2595344},
 {'date': '2024-02-27',
  'open': 296.8,
  'high': 296.8,
  'low': 293.32,
  'close': 293.76,
  'adjusted_close': 286.7912,
  'volume': 2811237},
 {'date': '2024-02-28',
  'open': 294.03,
  'high': 296.1,
  'low': 292.72,
  'close': 295.19,
  'adjusted_close': 288.1873,
  'volume': 2507286},
 {'date': '2024-02-29',
  'open': 294.2,
  'high': 294.75,
  'low': 291.71,
  'close': 292.28,
  'adjusted_close': 286.9698,


### Trying with Yahoo Finance to test

- Source: https://pythoninvest.com/long-read/exploring-finance-apis#popup:subscribe
- main documentation: https://pypi.org/project/yfinance/
- new documentation: https://ranaroussi.github.io/yfinance/index.html

In [6]:
mst = yf.Ticker("MSFT")

In [7]:
mst.info

HTTPError: 429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MSFT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true&crumb=Edge%3A+Too+Many+Requests

In [8]:
data = yf.download(['MSFT','GOOG', 'XXX'], period='1y')
data

[*********************100%%**********************]  3 of 3 completed


3 Failed downloads:
['GOOG', 'MSFT', 'XXX']: Exception('%ticker%: No price data found, symbol may be delisted (period=1y)')


Empty DataFrame
Columns: [(Adj Close, GOOG), (Adj Close, MSFT), (Adj Close, XXX), (Close, GOOG), (Close, MSFT), (Close, XXX), (High, GOOG), (High, MSFT), (High, XXX), (Low, GOOG), (Low, MSFT), (Low, XXX), (Open, GOOG), (Open, MSFT), (Open, XXX), (Volume, GOOG), (Volume, MSFT), (Volume, XXX)]
Index: []

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [9]:
data = yf.download(['MSFT','GOOG', 'XXX'], period='1y')['Adj Close']
data

[*********************100%%**********************]  3 of 3 completed


3 Failed downloads:
['MSFT', 'XXX', 'GOOG']: Exception('%ticker%: No price data found, symbol may be delisted (period=1y)')


,GOOG,MSFT,XXX
Date,,,


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [10]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
#def chunks(lst, n):
#    """Yield successive n-sized chunks from lst."""
#    for i in range(0, len(lst), n):
#        yield lst[i:i + n]   
        
#symbol_groups = list(chunks(stocks['Ticker'], 100))
#symbol_strings = []
#for i in range(0, len(symbol_groups)):
#    symbol_strings.append(','.join(symbol_groups[i]))
##     print(symbol_strings[i])

#my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [11]:
# create the a list from the stock file
# Join Tickers with commas
tickers_string = ','.join(stocks['Ticker'])

# strange output since the result count number of string
# len(tickers_string)

print(tickers_string)

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [12]:
# Batch call from yahoo Finance
data = yf.download(tickers_string, period='1y')['Adj Close']

# clean to remove NaN
data_cleaned = data.dropna(axis=1)
data_cleaned.reset_index(inplace=True)

# clean to remove NaN
data_cleaned = data.dropna(axis=1)
data_cleaned.reset_index(inplace=True)
data_cleaned

[                       1%%                      ]  3 of 505 completed

Failed to get ticker 'KSU' reason: Expecting value: line 1 column 1 (char 0)


[                       1%%                      ]  7 of 505 completed

Failed to get ticker 'FB' reason: Expecting value: line 1 column 1 (char 0)


[*                      2%%                      ]  10 of 505 completed

...

[**********************99%%**********************]  502 of 505 completed

Failed to get ticker 'GPS' reason: Expecting value: line 1 column 1 (char 0)


[*********************100%%**********************]  505 of 505 completed


505 Failed downloads:
['NOW', 'ODFL', 'HAL', 'TSN', 'SJM', 'FAST', 'BKR', 'IT', 'NEE', 'DTE', 'KSU', 'CTVA', 'VMC', 'FB', 'NUE', 'BXP', 'AMAT', 'PAYX', 'HSY', 'AIG', 'TIF', 'DOW', 'MDLZ', 'UDR', 'SCHW', 'URI', 'MAA', 'CINF', 'ATVI', 'NEM', 'ANET', 'MOS', 'ROL', 'WAT', 'MSI', 'BMY', 'ABC', 'ARE', 'INTC', 'AWK', 'LMT', 'CI', 'WM', 'CMA', 'SWK', 'DE', 'CXO', 'EQR', 'ETR', 'WAB', 'FDX', 'BSX', 'KLAC', 'HII', 'KMI', 'FMC', 'TFX', 'CTL', 'APD', 'MET', 'O', 'TMO', 'ADI', 'DISCK', 'TJX', 'TTWO', 'HIG', 'HUM', 'EVRG', 'VTR', 'WELL', 'KSS', 'DRI', 'FE', 'OTIS', 'MCK', 'PVH', 'NSC', 'PSA', 'KR', 'EFX', 'PYPL', 'BK', 'ATO', 'AVB', 'TPR', 'RE', 'MYL', 'TXT', 'WYNN', 'PAYC', 'ROP', 'BEN', 'HON', 'MGM', 'LW', 'HRB', 'EL', 'ANSS', 'EMN', 'HPE', 'FLS', 'WMT', 'EQIX', 'ECL', 'WY', 'ALLE', 'PFE', 'UAL', 'NVR', 'CSCO', 'LEN', 'IP', 'FIS', 'NKE', 'AXP', 'TRV', 'SPG', 'LKQ', 'MTB', 'WFC', 'ACN', 'HRL', 'ANTM', 'MLM', 'CARR', 'HLT', 'DAL', 'TXN', 'AIZ', 'AES', 'HST', 'F', 'CE', 'REGN', 'SRE', 'TMUS', 'PWR',

,Date,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS


In [13]:
# build different return signal 
n_length_index = len(data_cleaned)
series_last_day = np.array(data_cleaned.loc[n_length_index-1][1:])

# 1year return
series_1year_ago = np.array(data_cleaned.loc[0][1:])
return_1y = pd.Series(series_last_day / series_1year_ago - 1, name="1-year Return")

# 6 month return
# 252 days for a year thus 252/2 = 126 days for a semester (6 months)
series_6month_ago = np.array(data_cleaned.loc[n_length_index-127][1:])
return_6month = pd.Series(series_last_day / series_6month_ago - 1, name="6-month Return")

# 3 month return
# 252 days for a year thus 252/4 = 63 days for a quarter (3 months)
series_3month_ago = np.array(data_cleaned.loc[n_length_index-64][1:])
return_3month = pd.Series(series_last_day / series_3month_ago - 1, name="3-month Return")

# 1 month return
# 252 days for a year thus 252/12 = 21 days for a month
series_1month_ago = np.array(data_cleaned.loc[n_length_index-22][1:])
return_1month = pd.Series(series_last_day / series_1month_ago - 1, name="1-month Return")

# 1 week return
series_1week_ago = np.array(data_cleaned.loc[n_length_index-6][1:])
return_1week = pd.Series(series_last_day / series_1week_ago - 1, name="1-week Return")

# daily return
series_1day_ago = np.array(data_cleaned.loc[n_length_index-2][1:])
return_1day = pd.Series(series_last_day / series_1day_ago - 1, name="1-day Return")

data_returns = pd.concat([return_1day, return_1week, return_1month, 
                          return_3month, return_6month, return_1y], axis=1)
data_returns

KeyError: -1

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [ ]:
data_returns = data_returns = pd.concat([return_1day, return_1week, return_1month, 
                          return_3month, return_6month, return_1y], axis=1)
data_returns.index = data_cleaned.columns[1:]
final_data = data_returns.copy()
final_data

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [ ]:
# not optimal one by one loop
#for stock in final_data.index:
#    final_data.loc[stock,'1-day Return Percentile'] = stats.percentileofscore(
#        final_data['1-day Return'], final_data.loc[stock, '1-day Return'])
#final_data

In [ ]:
# Function to calculate percentile for each value in the series
def calculate_percentile(series):
    return series.apply(lambda x: stats.percentileofscore(series, x))/100

# Apply to multiple columns
final_data_percentiles = final_data.apply(calculate_percentile)

final_data_percentiles.columns = [col + ' Percentile' for col in final_data.columns]
final_data_percentiles

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [ ]:
final_data_percentiles['HQM Score'] = final_data_percentiles.mean(axis=1)
final_data_percentiles

## Selecting the 10 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [ ]:
final_data_complete = pd.concat([final_data, final_data_percentiles], axis=1)
final_data_complete['Stock Price'] = series_last_day
final_data_complete

In [ ]:
final_data_complete.sort_values('HQM Score', inplace=True, ascending=False)
final_data_complete

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
final_data_complete['Number of Shares to Buy'] = None
final_data_complete.head(12)

In [ ]:
#portfolio_size = input('Enter the value of your portfolio: ')
portfolio_size = 50_000

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number ! \nPlease try again")
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)
        
#portfolio_input()
print(portfolio_size)

In [ ]:
position_size = float(portfolio_size)/5
print(position_size)

In [ ]:
count_stock = 0
max_stock = 5
for i in final_data_complete.index:
    count_stock +=1
    if count_stock <= max_stock:
        #final_data_complete.loc[i, 'Number of Shares to Buy'] = math.floor(
        #    position_size / final_data_complete.loc[i, 'Stock Price'])
        final_data_complete.loc[i, 'Number of Shares to Buy'] = position_size / final_data_complete.loc[i, 'Stock Price']
    else:
        final_data_complete.loc[i, 'Number of Shares to Buy'] = 0

final_data_complete.head(12)

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
current_time_formatted_file = "".join(current_time_formatted.split(":"))
writer = pd.ExcelWriter(f'output/momentum_strategy_{current_time_formatted_file}.xlsx', engine='xlsxwriter')
final_data_complete.to_excel(writer, sheet_name = 'Momentum Strategy', index = True)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
    'A':['Ticker', string_template],
    'B':['1-day Return', percent_template],
    'C':['1-week Return', percent_template],
    'D':['1-month Return', percent_template],
    'E':['3-month Return', percent_template],
    'F':['6-month Return', percent_template],
    'G':['1-year Return', percent_template],
    'H':['1-day Return Percentile', percent_template],
    'I':['1-week Return Percentile', percent_template],
    'J':['1-month Return Percentile', percent_template],
    'K':['3-month Return Percentile', percent_template],
    'L':['6-month Return Percentile', percent_template],
    'M':['1-year Return Percentile', percent_template],
    'N':['HQM Score', percent_template],
    'O':['Stock Price', dollar_template],
    'P':['Number of Shares to Buy', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}',25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [ ]:
writer.close()

In [ ]:
# Stop the timer
end_time = time.time()

# Print the current date and time in a human-readable format
current_time = datetime.fromtimestamp(end_time)
print("Current date and time:", current_time.strftime("%Y-%m-%d %H:%M:%S"))

# Calculate the elapsed time
elapsed_time = end_time - start_time
# Format the elapsed time into hours, minutes, and seconds
hours, remainder = divmod(elapsed_time, 3600)
minutes, seconds = divmod(remainder, 60)

# Display the elapsed time
print(f"Total process time: {int(hours):02}:{int(minutes):02}:{int(seconds):02}")